In [2]:
import json
from tensorflow.keras import optimizers
import numpy as np

data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('testing.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  

for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('down')

down


In [3]:
en_num_data = [[en2id[en] for en in line ] for line in endata]
ch_num_data = [[ch2id[ch] for ch in line] for line in chdata]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in chdata]

print('char:', endata[1])
print('index:', en_num_data[1])
print(len(en_num_data))
len(ch_num_data)
# de_num_data

char: Choose a recorder.
index: [22, 55, 62, 62, 9, 65, 10, 8, 10, 11, 65, 38, 62, 11, 39, 65, 11, 42]
100


100

In [4]:
import numpy as np

max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 49
max decoder length: 11
index data:
 [22, 55, 62, 62, 9, 65, 10, 8, 10, 11, 65, 38, 62, 11, 39, 65, 11, 42]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250


In [20]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [21]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [22]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, None, 403)]  0                                            
__________________________________________________________________________________________________
lstm_12 (LSTM)                  [(None, None, 256),  330752      input_15[0][0]                   
__________________________________________________________________________________________________
lstm_14 (LSTM)                  [(None, None, 256),  675840      input_16[0][0]                   
                                                                 lstm_12[0][1]              

5/5 [==============================] - 1s 132ms/step - loss: 0.4717 - accuracy: 0.6764
Epoch 61/250
5/5 [==============================] - 1s 134ms/step - loss: 0.4665 - accuracy: 0.6782
Epoch 62/250
5/5 [==============================] - 1s 133ms/step - loss: 0.4617 - accuracy: 0.6800
Epoch 63/250
5/5 [==============================] - 1s 132ms/step - loss: 0.4632 - accuracy: 0.6809
Epoch 64/250
5/5 [==============================] - 1s 133ms/step - loss: 0.4593 - accuracy: 0.6809
Epoch 65/250
5/5 [==============================] - 1s 133ms/step - loss: 0.4594 - accuracy: 0.6755
Epoch 66/250
5/5 [==============================] - 1s 133ms/step - loss: 0.4626 - accuracy: 0.6755
Epoch 67/250
5/5 [==============================] - 1s 134ms/step - loss: 0.4541 - accuracy: 0.6809
Epoch 68/250
5/5 [==============================] - 1s 134ms/step - loss: 0.4591 - accuracy: 0.6791
Epoch 69/250
5/5 [==============================] - 1s 140ms/step - loss: 0.4580 - accuracy: 0.6800
Epoch 70/250


5/5 [==============================] - 1s 133ms/step - loss: 1.1698 - accuracy: 0.5336
Epoch 142/250
5/5 [==============================] - 1s 134ms/step - loss: 1.0376 - accuracy: 0.5800
Epoch 143/250
5/5 [==============================] - 1s 133ms/step - loss: 0.8654 - accuracy: 0.6055
Epoch 144/250
5/5 [==============================] - 1s 136ms/step - loss: 0.7499 - accuracy: 0.6336
Epoch 145/250
5/5 [==============================] - 1s 133ms/step - loss: 0.6460 - accuracy: 0.6418
Epoch 146/250
5/5 [==============================] - 1s 133ms/step - loss: 0.6319 - accuracy: 0.6436
Epoch 147/250
5/5 [==============================] - 1s 134ms/step - loss: 0.5967 - accuracy: 0.6500
Epoch 148/250
5/5 [==============================] - 1s 133ms/step - loss: 0.5414 - accuracy: 0.6573
Epoch 149/250
5/5 [==============================] - 1s 132ms/step - loss: 0.5340 - accuracy: 0.6600
Epoch 150/250
5/5 [==============================] - 1s 147ms/step - loss: 0.5331 - accuracy: 0.6555
Epoc

5/5 [==============================] - 1s 135ms/step - loss: 0.3351 - accuracy: 0.7009
Epoch 223/250
5/5 [==============================] - 1s 135ms/step - loss: 0.3482 - accuracy: 0.6955
Epoch 224/250
5/5 [==============================] - 1s 134ms/step - loss: 0.3435 - accuracy: 0.6964
Epoch 225/250
5/5 [==============================] - 1s 134ms/step - loss: 0.3315 - accuracy: 0.7027
Epoch 226/250
5/5 [==============================] - 1s 134ms/step - loss: 0.3235 - accuracy: 0.7073
Epoch 227/250
5/5 [==============================] - 1s 133ms/step - loss: 0.3151 - accuracy: 0.7064
Epoch 228/250
5/5 [==============================] - 1s 134ms/step - loss: 0.3031 - accuracy: 0.7064
Epoch 229/250
5/5 [==============================] - 1s 134ms/step - loss: 0.3044 - accuracy: 0.7073
Epoch 230/250
5/5 [==============================] - 1s 133ms/step - loss: 0.3007 - accuracy: 0.7109
Epoch 231/250
5/5 [==============================] - 1s 135ms/step - loss: 0.3021 - accuracy: 0.7073
Epoc

In [24]:
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])

In [ ]:
for k in range(0,100):
    test_data = encoder_input_data[k:k+1]
    h1, c1, h2, c2 = encoder_model.predict(test_data)
    target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
    target_seq[0, 0, ch2id['@']] = 1
    outputs = []
    while True:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq, h1, c1, h2, c2])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        outputs.append(sampled_token_index)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, sampled_token_index] = 1
        if sampled_token_index == ch2id['。']: break
        # if len(outputs) > 11: break
    
    print(endata[k])
    print(''.join([id2ch[i] for i in outputs]))


Look at these coasters over here.
你今天不许出击。
Choose a recorder.
四楼在卖亚麻布单。
I hadn't paid the telephone bill.
我给你指路。
That's easier said than done, of course.
这可是件大事儿啊！。
Side-to-Side Movements.
她着了魔。
about like 80 degrees.
你丫拉屎穿裤衩呀！。
We all are from Shandong.
俺都是山东人。
She was possessed by a devil.
她着了魔。
This wool knits up well.
这可是件大事儿啊！。
The majority was wrong last time.
这可是件大事儿啊！。
Stone Soup Stories to Go!
彩色玻璃窗板；。
Done. See you tomorrow.
我算得对吗。
He eased some of the strains on the poor.
你喜欢兔肉吗？。
Could it be that it was written wrongly?
四楼在卖亚麻布单。
What a terrible temper!
我的巴士在哪里？。
Great talents flower late.
大器晚成。
I forbid you to make a sortie today.
我给你指路。
C：My surname is Jiang.
四楼在卖亚麻布单。
Well, if it was greater .
俺都是山东人。
They looked over to the left.
这可是件大事儿啊！。
To supervise the management of printing industry.
问君能有几多愁？。
no one else can see you shake your head.
没人看的见你摇头。
All photos dials.
问君能有几多愁？。
Stained glass window panels;
彩色玻璃窗板；。
The murderer was caught red-handed.
这可是件大事儿啊！。
You don’t